# Wikipedia爬蟲練習
## 範例：練習是從Wikipedia中爬取文章。先定義一個搜尋的關鍵字，擷取該關鍵字詞的文章。

In [1]:
import requests
import re
from bs4 import BeautifulSoup

### 先定義一個我們想搜尋的字詞，並將它轉換成UTF-8編碼後的URL

In [2]:
input_keyword = "聖結石"  # 這裡可以自己定義有興趣的關鍵字

utf8_url = repr(input_keyword.encode('UTF-8')).upper()  # 編碼成UTF-8並轉成大寫字元
utf8_url = utf8_url.replace("\\X", "%")                 # 用 '%' 取代 '\X' 
print("%s: %s" % (input_keyword, utf8_url[2:-1:1]))     # 擷取中間的編碼結果

# 組成Wiki關鍵字搜尋的網址格式
root_keyword_link = '/wiki/' + utf8_url[2:-1:1]
print(root_keyword_link)

聖結石: %E8%81%96%E7%B5%90%E7%9F%B3
/wiki/%E8%81%96%E7%B5%90%E7%9F%B3


### 範例1：送出關鍵字請求後，爬取該關鍵字的文章內容

In [3]:
# 模擬封包的標頭
headers = {
    'authority': 'zh.wikipedia.org',
    'method': 'GET',
    'path': '/wiki/' + root_keyword_link,
    'scheme': 'https',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7,zh-CN;q=0.6',
    'cookie': 'GeoIP=TW:TPE:Taipei:25.05:121.53:v4; TBLkisOn=0; mwPhp7Seed=8b8; WMF-Last-Access-Global=04-Jun-2019; WMF-Last-Access=04-Jun-2019',
    'dnt': '1',
    #'if-modified-since': 'Tue, 04 Jun 2019 12:03:22 GMT',
    'referer': 'https://zh.wikipedia.org/wiki/Wikipedia:%E9%A6%96%E9%A1%B5',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36'
}    

url = 'https://zh.wikipedia.org' + root_keyword_link  # 組合關鍵字查詢URL
resp = requests.get(url, headers=headers)
resp.encoding = 'utf-8'

html = BeautifulSoup(resp.text, "lxml")
content = html.find(name='div', attrs={'id':'mw-content-text'}).find_all(name='p')

#
# 解析回傳資料，並萃取文章內容
#
for paragraph in content:
    print(paragraph.get_text())


聖結石（英語：Saint，1990年11月13日－），本名曾聖傑，為一位網路紅人、YouTuber，過去曾為頑Game成員，而後單飛並獨自經營YouTube頻道後吸引大量粉絲訂閱，稱之為「聖粉」。聖結石是目前台灣最快突破一百萬訂閱的YouTuber（225天）。





### 範例2：從爬取的文章內容中，擷取出有外部連結的關鍵字。這些關鍵字在文章中是以藍色字體顯示，會連到外部的網頁，並解釋其內容。

In [4]:
for ext_link in content:
    a_tag = ext_link.find_all('a', href=re.compile("^(/wiki/)((?!;)\S)*$"))
    if len(a_tag) > 0:
        for link_string in a_tag:
            a_link = link_string["href"]       # 外部連結的網址
            a_keyword = link_string.get_text()  # 外部連結的中文名稱
            print("外部連結: [%s] %s" % (a_keyword, a_link))

外部連結: [網路紅人] /wiki/%E7%B6%B2%E7%B5%A1%E7%B4%85%E4%BA%BA
外部連結: [YouTuber] /wiki/YouTuber
外部連結: [台灣] /wiki/%E5%8F%B0%E7%81%A3


## 作業：接下來定義一個爬蟲函數，這個函數的主要工作為：
### (1) 爬取當前關鍵字的解釋，並存入檔案(因為文章內容太多會佔滿整個頁面，所以存程檔案，方便後續檢視)
### (2) 萃取出當前關鍵字所引用的外部連結，當作新的查詢關鍵字
### (3) 把第(2)擷取到的關鍵字當作新的關鍵字，回到第(1)步，爬取新的關鍵字解釋。

In [5]:
import os.path
def WikiArticle(key_word_link, key_word, recursive):
    
    if (recursive <= max_recursive_depth):
        print("遞迴層[%d] - %s (%s)" % (recursive, key_word_link, key_word))
        
        # 模擬封包的標頭
        headers = {
            'authority': 'zh.wikipedia.org',
            'method': 'GET',
            'path': '/wiki/' + key_word_link,
            'scheme': 'https',
            'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
            'accept-encoding': 'gzip, deflate, br',
            'accept-language': 'zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7,zh-CN;q=0.6',
            'cookie': 'GeoIP=TW:TPE:Taipei:25.05:121.53:v4; TBLkisOn=0; mwPhp7Seed=8b8; WMF-Last-Access-Global=04-Jun-2019; WMF-Last-Access=04-Jun-2019',
            'dnt': '1',
            #'if-modified-since': 'Tue, 04 Jun 2019 12:03:22 GMT',
            'referer': 'https://zh.wikipedia.org/wiki/Wikipedia:%E9%A6%96%E9%A1%B5',
            'upgrade-insecure-requests': '1',
            'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36'
        }    

        url = 'https://zh.wikipedia.org' + key_word_link  # 組合關鍵字查詢URL
        resp = requests.get(url, headers=headers)
        resp.encoding = 'utf-8'

        html = BeautifulSoup(resp.text, "lxml")
        content = html.find(name='div', attrs={'id':'mw-content-text'}).find_all(name='p')
        
        #
        # Part 1: 請參考範例1，爬取當前關鍵字的文章內容。
        #         因為內容太多，我們把它寫入檔案，並以關鍵字作為檔案名稱，以便稍後查閱內容。
        #         請先建立一個名為"WikiArticle"的資料夾，爬取到的文章內容會放在這個資料夾底下。
        #
        save_path='./WikiArticle/'
        if not os.path.exists(save_path):
            os.makedirs(save_path)
        completeName=os.path.join(save_path, key_word+".txt")
        file1 = open(completeName, "w")
        toWrite=''
        for paragraph in content:
            try:
                file1.write(toWrite)
            except UnicodeEncodeError:
                print(paragraph)
        file1.close()

        
        #
        # Part 2: 請參考範例2，萃取出本篇文章中所延伸引用的外部連結，並儲存在external_link_dict
        #
        external_link_dict = dict({})
        for ext_link in content:
            a_tag = ext_link.find_all('a', href=re.compile("^(/wiki/)((?!;)\S)*$"))
            if len(a_tag) > 0:
                for link_string in a_tag:
                    external_link_dict[link_string["href"]]=link_string.get_text()

                    
        #
        # Part 3: 將Part 2所收集的外部連結，當作新的關鍵字，繼續迭代深入爬蟲
        #
        if (len(external_link_dict) > 0):
            recursive += 1  # 遞迴深度加1
            for k, v in external_link_dict.items():
                WikiArticle(k, v, recursive)  # 再次呼叫同樣的函數，執行同樣的流程
                

### 執行前個步驟定義好的爬蟲主程式

In [6]:
# 定義爬取的遞迴深度。深度不要訂太深，否則會爬很久。
max_recursive_depth = 2

WikiArticle(root_keyword_link, input_keyword, 0)

遞迴層[0] - /wiki/%E8%81%96%E7%B5%90%E7%9F%B3 (聖結石)
遞迴層[1] - /wiki/%E7%B6%B2%E7%B5%A1%E7%B4%85%E4%BA%BA (網路紅人)
遞迴層[2] - /wiki/%E7%B6%B2%E9%9A%9B%E7%B6%B2%E8%B7%AF (網路)
遞迴層[2] - /wiki/%E7%A4%BE%E7%BE%A4%E7%B6%B2%E7%AB%99 (社群網站)
遞迴層[2] - /wiki/%E5%BD%B1%E9%9F%B3%E7%B6%B2%E7%AB%99 (影音網站)
遞迴層[2] - /wiki/%E8%99%9B%E6%93%AC%E7%A4%BE%E7%BE%A4 (虛擬社群)
遞迴層[2] - /wiki/%E7%B6%B2%E8%B7%AF%E7%88%86%E7%B4%85 (網路爆紅)
遞迴層[2] - /wiki/%E7%B6%B2%E8%AA%8C (部落格)
遞迴層[2] - /wiki/%E5%80%8B%E6%80%A7 (個性)
遞迴層[2] - /wiki/%E7%B2%89%E7%B5%B2 (粉絲)
遞迴層[2] - /wiki/%E5%B7%A5%E4%BD%9C%E5%AE%A4 (工作室)
遞迴層[2] - /wiki/%E7%B6%93%E7%B4%80%E5%85%AC%E5%8F%B8 (經紀公司)
遞迴層[2] - /wiki/%E5%93%81%E7%89%8C (品牌)
遞迴層[2] - /wiki/%E7%94%A2%E5%93%81 (產品)
遞迴層[2] - /wiki/%E5%A8%9B%E6%A8%82%E5%9C%88 (娛樂圈)
遞迴層[2] - /wiki/%E8%97%9D%E4%BA%BA (藝人)
遞迴層[2] - /wiki/%E7%B4%A0%E4%BA%BA (素人)
遞迴層[2] - /wiki/%E7%82%92%E4%BD%9C (炒作)
遞迴層[2] - /wiki/%E6%A5%AD%E9%85%8D (業配)
遞迴層[2] - /wiki/%E5%96%AE%E9%A0%81%E9%BB%9E%E9%96%B1%E7%8E%87 (點閱率)
遞迴層[2] - /wiki/%E9%BB%9

遞迴層[2] - /wiki/%E8%91%A1%E8%90%84%E7%89%99 (葡萄牙)
遞迴層[2] - /wiki/%E5%B1%B1%E6%B5%B7%E7%B6%93 (山海經)
遞迴層[2] - /wiki/%E8%93%AC%E8%90%8A (蓬萊)
遞迴層[2] - /wiki/%E7%80%9B%E6%B4%B2 (瀛洲)
遞迴層[2] - /wiki/%E5%9C%B0%E5%BD%A2 (地形)
遞迴層[2] - /wiki/%E9%AF%A4%E5%B3%B6 (鯤島)
遞迴層[2] - /wiki/%E5%8D%AB%E6%B8%A9 (衛溫)
遞迴層[2] - /wiki/%E8%AB%B8%E8%91%9B%E7%9B%B4 (諸葛直)
遞迴層[2] - /wiki/%E5%AD%AB%E5%90%B3 (孫吳)
遞迴層[2] - /wiki/%E5%A4%B7%E6%B4%B2 (夷洲)
遞迴層[2] - /wiki/%E6%97%A5%E6%9C%AC (日本)
遞迴層[2] - /wiki/%E9%87%91%E5%9C%B0%E9%99%A2 (金地院)
遞迴層[2] - /wiki/%E7%89%87%E5%81%87%E5%90%8D (片假名)
遞迴層[2] - /wiki/%E6%BC%A2%E5%AD%97 (漢字)
遞迴層[2] - /wiki/%E7%B9%81%E9%AB%94%E5%AD%97 (正體字)
遞迴層[2] - /wiki/%E7%95%B0%E9%AB%94%E5%AD%97 (異體字)
遞迴層[2] - /wiki/%E8%87%BA%E7%81%A3%E6%94%BF%E5%BA%9C (政府)
遞迴層[2] - /wiki/%E6%96%87%E4%B9%A6 (文書)
遞迴層[2] - /wiki/%E8%87%BA%E5%8C%97%E5%B8%82%E6%94%BF%E5%BA%9C (臺北市政府)
遞迴層[2] - /wiki/%E9%A6%AC%E8%8B%B1%E4%B9%9D (馬英九)
遞迴層[2] - /wiki/%E8%87%BA%E5%8C%97%E5%B8%82%E5%B8%82%E9%95%B7 (市長)
遞迴層[2] - /wiki/%E4%B8%AD%E

遞迴層[2] - /wiki/%E8%87%BA%E7%81%A3%E8%BB%8D%E4%BA%8B (軍事)
遞迴層[2] - /wiki/%E8%87%BA%E7%81%A3%E5%A4%96%E4%BA%A4 (外交)
遞迴層[2] - /wiki/%E8%88%8A%E7%9F%B3%E5%99%A8%E6%99%82%E4%BB%A3 (舊石器時代)
遞迴層[2] - /wiki/%E6%BE%8E%E6%B9%96%E6%B0%B4%E9%81%93 (澎湖水道)
遞迴層[2] - /wiki/%E6%BE%8E%E6%B9%96%E5%8E%9F%E4%BA%BA (澎湖原人)
遞迴層[2] - /wiki/%E6%96%B0%E7%9F%B3%E5%99%A8%E6%99%82%E4%BB%A3 (新石器時代)
遞迴層[2] - /wiki/%E5%8D%97%E5%B3%B6%E8%AA%9E%E7%B3%BB (南島語系)
遞迴層[2] - /wiki/%E8%87%BA%E6%9D%B1%E7%B8%A3 (臺東)
遞迴層[2] - /wiki/%E9%95%B7%E6%BF%B1%E6%96%87%E5%8C%96 (長濱文化)
遞迴層[2] - /wiki/%E8%8B%97%E6%A0%97%E7%B8%A3 (苗栗縣)
遞迴層[2] - /wiki/%E8%87%BA%E5%8D%97%E5%B8%82 (臺南)
遞迴層[2] - /wiki/%E5%B7%A6%E9%8E%AE%E4%BA%BA (左鎮人)
遞迴層[2] - /wiki/%E5%A4%A7%E5%9D%8C%E5%9D%91%E6%96%87%E5%8C%96 (大坌坑文化)
遞迴層[2] - /wiki/%E8%87%BA%E5%8C%97%E5%B8%82 (臺北市)
遞迴層[2] - /wiki/%E5%9C%93%E5%B1%B1%E6%96%87%E5%8C%96 (圓山文化)
遞迴層[2] - /wiki/%E7%87%9F%E5%9F%94%E6%96%87%E5%8C%96 (營埔文化)
遞迴層[2] - /wiki/%E5%8D%91%E5%8D%97%E6%96%87%E5%8C%96 (卑南文化)
遞迴層[2] - /wiki/%E5%8D%8

UnicodeEncodeError: 'latin-1' codec can't encode characters in position 76-83: ordinal not in range(256)